In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# DON'T FORGET TO ACTIVATE THE GPU when on google colab (Edit > Notebook settings)
from os import environ
GOOGLE_COLAB = True if "COLAB_GPU" in environ else False
if GOOGLE_COLAB:
    !pip install git+https://github.com/undark-lab/swyft.git

In [3]:
import numpy as np
import pylab as plt
import swyft

In [4]:
device = 'cuda' if swyft.utils.is_cuda_available() else "cpu"
n_training_samples = 3000
n_parameters = 2
marginal_indices = list(range(n_parameters))
observation_key = "x"

In [5]:
def model(v, sigma = 0.2):
    x = v + np.random.randn(n_parameters) * sigma
    return {observation_key: x}

v_o = np.zeros(n_parameters)
observation_o = model(v_o, sigma = 0.)

n_observation_features = observation_o[observation_key].shape[0]
observation_shapes = {key: value.shape for key, value in observation_o.items()}

In [6]:
simulator = swyft.Simulator(
    model, 
    n_parameters, 
    sim_shapes={observation_key: (n_parameters,)}
)
store = swyft.MemoryStore(simulator)

Creating new store.


In [7]:
low = -1 * np.ones(n_parameters)
high = 1 * np.ones(n_parameters)
prior = swyft.get_uniform_prior(low, high)
store.add(n_training_samples, prior)
store.simulate()

Store: Adding 2934 new samples to simulator store.


In [8]:
dataset = swyft.Dataset(n_training_samples, prior, store)

In [9]:
from swyft.inference.marginalratioestimator import MarginalRatioEstimator
from swyft.networks.classifier import get_marginal_classifier

In [10]:
network = get_marginal_classifier(
    observation_key=observation_key,
    marginal_indices=marginal_indices,
    observation_shapes=observation_shapes,
    n_parameters=n_parameters,
    hidden_features=32,
    num_blocks=2,
)

mre = MarginalRatioEstimator(
    marginal_indices=marginal_indices,
    network=network,
    device=device,
)

In [11]:
mre.train(dataset)

training: lr=5e-07, epoch=25, validation loss=1.759

## old

In [12]:
post = swyft.Posteriors(dataset)

In [13]:
post.add([(0, 1)], device=DEVICE)
post.train([(0, 1)], max_epochs = 20)

NameError: name 'DEVICE' is not defined

In [ ]:
samples = post.sample(1000000, obs0)
swyft.plot_corner(samples, [0, 1], color='k', figsize = (8,8), truth=v0, bins = 40);